In [1]:
# imports
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from pandarallel import pandarallel

In [2]:
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
### declare constants
TIMEPOINTS = [f'ZT{i}' for i in range(0,23) if i%2==0]
TIMEPOINTS

['ZT0',
 'ZT2',
 'ZT4',
 'ZT6',
 'ZT8',
 'ZT10',
 'ZT12',
 'ZT14',
 'ZT16',
 'ZT18',
 'ZT20',
 'ZT22']

In [4]:
df = pd.read_parquet("./data_files/concatenated_data/gatk_all_samples_concatenated_annt_added_multiline_dupes_dropped.parquet")

In [5]:
df.head(3)

,tissue,timepoint,loci,refCount,altCount,totalCount,refBias,binomTest,fdr,REF,ALT,ANN[*].ALLELE,ANN[*].EFFECT,ANN[*].IMPACT,ANN[*].GENE,ANN[*].FEATURE,ANN[*].BIOTYPE
0,HIP,ZT0,NC_044976.1:204,1,17,18,0.055556,0.000145,0.012577,T,A,.,.,.,.,.,.
1,HIP,ZT0,NC_044976.1:4126,1,0,1,1.000000,1.000000,1.000000,G,A,.,.,.,.,.,.
2,HIP,ZT0,NC_044976.1:4522,0,1,1,0.000000,1.000000,1.000000,G,A,.,.,.,.,.,.


In [6]:
df['refBias'] = df['refBias'].astype(float)

In [7]:
df['uq_id'] = df['tissue'] + ":" + df['timepoint'] + ":" + df['loci']

In [8]:
df.shape

(47889413, 18)

In [9]:
## read snpeff vcf file, create a dataframe, merge with AI file
for tp in TIMEPOINTS:
    print(tp)
    phase_dat = pd.read_csv(f"../../wasp_rerun_june22/data_files/vcfs_annotated_with_snpeff_all/{tp}_snpeff.vcf", sep="\t", comment="#", header=None)
    phase_dat['loci'] = phase_dat[0] + ":" + phase_dat[1].astype(str)
    phase_dat = phase_dat[['loci', 8, 9]]
    phase_dat.columns = ['loci', 'phase_info', 'phase_dat']
    tmp = df[df['timepoint']==tp].merge(phase_dat, on='loci', how='left')
    if tp == "ZT0":
        dat = tmp
    else:
        dat = dat.append(tmp)

ZT0
ZT2
ZT4
ZT6
ZT8
ZT10
ZT12
ZT14
ZT16
ZT18
ZT20
ZT22


In [10]:
dat.sample(50)

,tissue,timepoint,loci,refCount,altCount,totalCount,refBias,binomTest,fdr,REF,ALT,ANN[*].ALLELE,ANN[*].EFFECT,ANN[*].IMPACT,ANN[*].GENE,ANN[*].FEATURE,ANN[*].BIOTYPE,uq_id,phase_info,phase_dat
465047,VIC,ZT16,NC_044993.1:66599840,0,1,1,0.000000,1.000000e+00,1.000000e+00,G,A,A,intron_variant,MODIFIER,RBFOX1,transcript,protein_coding,VIC:ZT16:NC_044993.1:66599840,GT:AD:DP:GQ:PS,"0|1:11,26:37:99:63620783"
569331,HEA,ZT8,NC_044988.1:132065,2,1,3,0.666667,1.000000e+00,1.000000e+00,A,G,G,non_coding_transcript_exon_variant,MODIFIER,LOC101017094,transcript,pseudogene,HEA:ZT8:NC_044988.1:132065,GT:AD:DP:GQ:PS,"0|1:18,22:40:99:99768"
4130166,KIC,ZT18,NC_044994.1:66356799,0,1,1,0.000000,1.000000e+00,1.000000e+00,A,G,.,.,.,.,.,.,KIC:ZT18:NC_044994.1:66356799,GT:AD:DP:GQ:PS,"0|1:19,21:40:99:40432080"
2632912,ILE,ZT8,NC_044981.1:126429485,0,1,1,0.000000,1.000000e+00,1.000000e+00,G,C,.,.,.,.,.,.,ILE:ZT8:NC_044981.1:126429485,GT:AD:DP:GQ:PS,"0|1:22,18:40:99:4644292"
3159509,ILE,ZT14,NC_044982.1:125448073,2,2,4,0.500000,1.000000e+00,1.000000e+00,G,A,A,intron_variant,MODIFIER,DAPK2,transcript,protein_coding,ILE:ZT14:NC_044982.1:125448073,GT:AD:DP:GQ:PS,"0|1:19,26:45:99:81813632"
44416,SPL,ZT18,NC_044976.1:204452726,2,1,3,0.666667,1.000000e+00,1.000000e+00,T,C,.,.,.,.,.,.,SPL:ZT18:NC_044976.1:204452726,GT:AD:DP:GQ:PS,"1|0:25,26:52:99:204435720"
3296803,ILE,ZT10,NC_044976.1:25997598,2,3,5,0.400000,1.000000e+00,1.000000e+00,T,G,G,intron_variant,MODIFIER,IFI6,transcript,protein_coding,ILE:ZT10:NC_044976.1:25997598,GT:AD:DP:GQ:PS,"1|0:15,20:36:99:25670064"
1863842,VIC,ZT0,NC_044989.1:61802274,1,0,1,1.000000,1.000000e+00,1.000000e+00,A,G,G,intron_variant,MODIFIER,USP34,transcript,protein_coding,VIC:ZT0:NC_044989.1:61802274,GT:AD:DP:GQ:PS,"1|0:13,17:30:99:61636554"
4052749,VIC,ZT22,NC_044989.1:10612341,5,3,8,0.625000,7.265625e-01,1.000000e+00,C,T,T,intron_variant,MODIFIER,PDIA6,transcript,protein_coding,VIC:ZT22:NC_044989.1:10612341,GT:AD:DP:GQ:PS,"1|0:20,17:37:99:9860043"
3562254,LUN,ZT8,NC_044987.1:107667494,6,3,9,0.666667,5.078125e-01,1.000000e+00,G,A,A,intron_variant,MODIFIER,SSRP1,transcript,protein_coding,LUN:ZT8:NC_044987.1:107667494,GT:AD:DP:GQ:PS,"1|0:23,20:45:99:106605127"


In [11]:
## function to check whether the SNV is phased
def is_phased(x):
    if pd.isnull(x):
        return "no_data"
    else:
        x = x.split(":")
        if "PS" in x:
            return 'phased'
        else:
            return 'not_phased'

In [12]:
## label each SNV as phased or unphased
dat['is_phased'] = dat['phase_info'].parallel_apply(is_phased)
dat['is_phased'].value_counts()
####################################

phased        47751036
not_phased      138377
Name: is_phased, dtype: int64

In [13]:
## Get phase ID for each SNV
def get_phased_block_id(x: pd.Series):
    if x['is_phased'] == 'no_data':
        return 'no_data'
    else:
        phase_data = {i:j for i, j in zip(x['phase_info'].split(":"), x['phase_dat'].split(":"))}
        if 'PS' in phase_data:
            return phase_data['PS']
        else:
            return 'not_phased'

In [14]:
dat['phase_id'] = dat.parallel_apply(get_phased_block_id, axis=1)

In [15]:
def compute_homoloBias(x: pd.Series):
    if x['is_phased']=="no_data":
        return "no_data"
    else:
        phase_data = {i:j for i, j in zip(x['phase_info'].split(":"), x['phase_dat'].split(":"))}
        if 'PS' not in phase_data:
            return np.nan
        elif phase_data['GT'] == '0|1':
            return x['refBias']
        elif phase_data['GT'] == '1|0':
            return 1-x['refBias']
        else:
            raise ValueError

In [16]:
dat['homologBias'] = dat.parallel_apply(compute_homoloBias, axis=1)

In [17]:
dat.to_parquet("./data_files/concatenated_data/gatk_all_samples_concatenated_annt_added_multiline_dupes_dropped_phase_added.parquet")